# ⚡ Germany Electricity Market Analysis (2019–2024)
### Data Engineering Pipeline

**Project Objective:**
To aggregate and sanitize wholesale electricity data from the SMARD platform (Bundesnetzagentur) for downstream analysis of price drivers, residual load dynamics, and market volatility.

**Methodology:**
1.  **Ingestion:** Loading raw CSVs with German-specific formatting (`.` thousands, `,` decimals).
2.  **Alignment:** Merging Consumption, Generation, and Price datasets on strict timestamps.
3.  **Validation:** Statistical audit to ensure data integrity before Tableau export.

In [ ]:
import pandas as pd
import numpy as np
import os

# ---------------------------------------------------------
# CONFIGURATION
# ---------------------------------------------------------
# Using relative paths ensures this code works on any machine (Windows/Mac/Linux)
DATA_RAW_DIR = os.path.join("..", "data", "raw", "smard")
DATA_PROCESSED_DIR = os.path.join("..", "data", "processed")

# Create output directory if it doesn't exist
os.makedirs(DATA_PROCESSED_DIR, exist_ok=True)

# ---------------------------------------------------------
# HELPER FUNCTIONS
# ---------------------------------------------------------
def load_and_standardize(filename, sep=';'):
    """
    Loads SMARD csv files handling German numeric formatting.
    Parses dates robustly, handling both 'DD.MM.YYYY' and 'Jan 1, 2019' formats.
    """
    path = os.path.join(DATA_RAW_DIR, filename)
    try:
        # Read the CSV
        df = pd.read_csv(path, sep=sep, thousands=',', decimal='.')
        
        # Standardize Date Columns
        for col in ['Start date', 'End date']:
            if col in df.columns:
                # 'format="mixed"' allows Pandas to handle variations automatically.
                # dayfirst=True ensures correct parsing for European dates (e.g., 01/02 is Feb 1st)
                # unless the format explicitly uses month names (like "Jan").
                try:
                    df[col] = pd.to_datetime(df[col], format='mixed', dayfirst=True)
                except Exception as e:
                    print(f"⚠️ Date parsing warning in {filename} column {col}: {e}")
            
        return df
    except FileNotFoundError:
        print(f"❌ Error: File not found at {path}")
        return None

def clean_energy_data(df):
    """
    Coerces non-date columns to numeric, handling missing '-' strings as NaN.
    """
    if df is None: return None
    
    # Identify columns that are NOT dates
    cols_to_fix = [c for c in df.columns if c not in ['Start date', 'End date']]
    
    # Force numeric conversion (errors='coerce' turns '-' into NaN)
    df[cols_to_fix] = df[cols_to_fix].apply(pd.to_numeric, errors='coerce')
    
    return df

# 1. Data Ingestion & Standardization
**Objective:** Ingest raw SMARD market data.

We process the following datasets:
* **Consumption:** Realized grid load.
* **Generation:** Realized generation by source (Wind, Solar, Coal, Gas, etc.).
* **Prices:** Day-ahead wholesale electricity prices.

In [39]:
# 1. Load Data
print("Loading datasets...")
consumption = load_and_standardize('consumption.csv')
generation = load_and_standardize('generation.csv')
prices = load_and_standardize('prices.csv')

# 2. Clean Data (Handle numeric conversions)
consumption = clean_energy_data(consumption)
generation = clean_energy_data(generation)
prices = clean_energy_data(prices)

print("✅ Data loaded and types standardized.")

Loading datasets...
✅ Data loaded and types standardized.


# 2. Temporal Alignment & Merging
**Objective:** Create a unified "Master Table" by aligning consumption, generation, and pricing data.

We utilize an **Inner Join** on both `Start date` and `End date`.
* **Why both dates?** Ensures matching of exact settlement periods (00:00–00:15).
* **Why Inner Join?** Filters for periods where *all* three data points exist, eliminating incomplete records.

In [40]:
# Merge 1: Consumption + Generation
energy_data = consumption.merge(
    generation, 
    on=['Start date', 'End date'], 
    how='inner', 
    suffixes=('_cons', '_gen')
)

# Merge 2: Add Prices
energy_data = energy_data.merge(
    prices, 
    on=['Start date', 'End date'], 
    how='inner'
)

print(f"✅ Merge complete. Combined shape: {energy_data.shape}")

✅ Merge complete. Combined shape: (2192, 35)


# 3. Quality Assurance & Export
**Objective:** Validate dataset integrity before committing to the visualization layer.

We perform a programmatic audit:
1.  **Coverage Check:** Verifying the time horizon covers the expected range.
2.  **Null Analysis:** Quantifying missing values.
3.  **Feature Renaming:** Mapping verbose SMARD column names to clean, SQL-friendly names for Tableau.

In [41]:
# ---------------------------------------------------------
# A. Quality Assurance (QA)
# ---------------------------------------------------------

# 1. Temporal Coverage
print(f"Dataset coverage: {energy_data['Start date'].min()} to {energy_data['Start date'].max()}")

# 2. Duplicate Check (Crucial for Time-Series)
duplicates = energy_data.duplicated(subset=["Start date", "End date"]).sum()
if duplicates > 0:
    print(f"⚠️ WARNING: Found {duplicates} duplicate entries in time index.")
else:
    print("✅ Primary key check passed (No duplicates).")

# 3. Missing Value Audit
print("\nMissing values found (Count):")
missing_count = energy_data.isnull().sum()
print(missing_count[missing_count > 0])

# 4. Statistical Sanity Check
print("\nKey Statistics (Transposed):")
# We use display() if in Jupyter, otherwise print()
try:
    display(energy_data.describe().T.head(10)) # Showing top 10 columns to save space
except:
    print(energy_data.describe().T.head(10))

# ---------------------------------------------------------
# B. Feature Engineering / Renaming
# ---------------------------------------------------------
# Standardizing column names for easier use in Tableau/SQL
rename_map = {
    "grid load [MWh] Calculated resolutions": "grid_load_mwh",
    "Residual load [MWh] Calculated resolutions": "residual_load_mwh",
    "Germany/Luxembourg [€/MWh] Calculated resolutions": "price_de_lu_eur"
}
energy_data.rename(columns=rename_map, inplace=True)
print(f"\n✅ Columns renamed for Tableau compatibility.")

# ---------------------------------------------------------
# C. Export
# ---------------------------------------------------------
output_file = os.path.join(DATA_PROCESSED_DIR, "energy_market_merged.csv")
energy_data.to_csv(output_file, index=False)

print("-" * 30)
print(f"🚀 SUCCESS: Processed data saved to:\n{os.path.abspath(output_file)}")
print("-" * 30)

Dataset coverage: 2019-01-01 00:00:00 to 2024-12-31 00:00:00
✅ Primary key check passed (No duplicates).

Missing values found (Count):
Nuclear [MWh] Calculated resolutions                 1903
∅ DE/LU neighbours [€/MWh] Calculated resolutions     323
Poland [€/MWh] Calculated resolutions                 323
DE/AT/LU [€/MWh] Calculated resolutions              2192
dtype: int64

Key Statistics (Transposed):


,count,mean,min,25%,50%,75%,max,std
Start date,2192,2021-12-31 12:00:00,2019-01-01 00:00:00,2020-07-01 18:00:00,2021-12-31 12:00:00,2023-07-02 06:00:00,2024-12-31 00:00:00,NaN
End date,2192,2022-01-01 12:00:00,2019-01-02 00:00:00,2020-07-02 18:00:00,2022-01-01 12:00:00,2023-07-03 06:00:00,2025-01-01 00:00:00,NaN
grid load [MWh] Calculated resolutions,2192.0,1320030.33474,864718.0,1218312.625,1325750.0,1435447.1875,1681885.5,161068.874357
Grid load incl. hydro pumped storage [MWh] Calculated resolutions,2192.0,1353526.648038,906499.5,1252975.3125,1357655.0,1466321.4375,1723689.75,158739.201053
Hydro pumped storage [MWh] Calculated resolutions_cons,2192.0,33496.313298,8523.25,25930.3125,32700.5,39937.8125,70895.25,10707.801807
Residual load [MWh] Calculated resolutions,2192.0,826288.004165,93334.5,650255.625,834050.25,1014454.25,1568968.25,258576.425547
Biomass [MWh] Calculated resolutions,2192.0,104430.008554,87926.75,99781.8125,104486.75,109264.1875,118377.25,6330.24601
Hydropower [MWh] Calculated resolutions,2192.0,42281.697422,22086.25,34594.375,42322.125,49219.0,65611.5,9357.303699
Wind offshore [MWh] Calculated resolutions,2192.0,68070.965785,179.0,32847.75,66492.75,101911.625,159836.25,39433.677602
Wind onshore [MWh] Calculated resolutions,2192.0,285095.5276,7213.75,122099.5,224969.375,400222.75,1060473.5,211023.351644



✅ Columns renamed for Tableau compatibility.
------------------------------
🚀 SUCCESS: Processed data saved to:
/Users/gauravhada/Downloads/projects/germany_energy_project/data/processed/energy_market_merged.csv
------------------------------
